In [1]:
# first convert ods to csv

import pandas as pd
import pickle
from aksharamukha import transliterate
import re

read in inflections table

In [ ]:
inflection_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection-generator/declensions & conjugations.xlsx', sheet_name="declensions", dtype=str)

inflection_df = inflection_df.shift(periods=2)

inflection_df.columns = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "AC", "AD", "AE", "AF", "AG", "AH", "AI", "AJ", "AK", "AL", "AM", "AN", "AO", "AP", "AQ", "AR", "AS", "AT", "AU", "AV", "AW", "AX", "AY", "AZ", "BA", "BB", "BC", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BK", "BL", "BM", "BN", "BO", "BP", "BQ", "BR", "BS", "BT", "BU", "BV", "BW", "BX", "BY", "BZ", "CA", "CB", "CC", "CD", "CE", "CF", "CG", "CH", "CI", "CJ", "CK", "CL", "CM", "CN", "CO", "CP", "CQ", "CR", "CS", "CT", "CU", "CV", "CW", "CX", "CY", "CZ", "DA", "DB", "DC", "DD", "DE", "DF", "DG", "DH", "DI", "DJ", "DK"]
inflection_df.fillna("", inplace=True)

read index

In [ ]:
index_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection-generator/declensions & conjugations.xlsx', sheet_name="index", dtype=str)
index_df.fillna("", inplace=True)
# index_df["inflection name"] = index_df["inflection name"].str.replace(" ", "_") #only needed if using as variable
index_df_length = len(index_df)

generate df's of each inflection pattern

In [ ]:
import re

for row in range(index_df_length):
	inflection_name = index_df.iloc[row,0]
	cell_range = index_df.iloc[row,1]
	like = index_df.iloc[row,2]
	irreg = index_df.iloc[row,3]
	
	col_range_1 = re.sub("(.+?)\d*\:.+", "\\1", cell_range)
	col_range_2 = re.sub(".+\:(.[A-Z]*)\d*", "\\1", cell_range)
	row_range_1 = int(re.sub(".+?(\d{1,3}):.+", "\\1", cell_range))
	row_range_2 = int(re.sub(".+:.+?(\d{1,3})", "\\1", cell_range))

	# print (f"{inflection_name} || {cell_range} || {col_range_1}:{col_range_2} || {row_range_1}:{row_range_2}")

	inflection_df_filtered = inflection_df.loc[row_range_1:row_range_2, col_range_1:col_range_2]
	inflection_df_filtered.Name =  f"{inflection_name}" #help! change the name to "inflection name" 

	inflection_df_filtered.to_csv(f"output/patterns/{inflection_name}.csv", sep="\t", index=False, header=False)

generate string of all inflections for each headword

In [ ]:
# dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd.csv", sep="\t", dtype=str)
# dpd_df.fillna("", inplace=True)
# dpd_df_length = len(dpd_df)

# all_inflections = ""

# for row in range(dpd_df_length): #dpd_df_length
# 	headword = dpd_df.loc[row, "Pāli1"]
# 	headword_clean = re.sub(" \d*$", "", headword)
# 	stem = dpd_df.loc[row, "Stem"]
# 	if stem == "*":
# 		stem = ""
# 	pattern = dpd_df.loc[row, "Pattern"]
# 	pos = dpd_df.loc[row, "POS"]
# 	metadata = dpd_df.loc[row, "Metadata"]
# 	meaning = dpd_df.loc[row, "Meaning IN CONTEXT"]

# 	if row % 2000 == 0:
# 		print(f"{row} {headword}")

# 	if pos != "prefix" and pos != "cs" and pos != "suffix" and pos != "ve" and pos != "idiom" and meaning != "" and metadata != "yes":
# 		# print(f"{headword=} {headword_clean=} {stem=} {pattern=} {pos=} {metadata=} {meaning=}")

# 		if stem == "-":
# 			all_inflections += headword_clean + " "

# 		elif stem == "!":
# 			all_inflections += headword_clean + " "

# 		else:
# 			try:
# 				df = pd.read_csv(f"output/patterns/{pattern}.csv", sep="\t", header=None)
# 				df.fillna("", inplace=True)
# 				df_rows = df.shape[0]
# 				df_columns = df.shape[1]

# 				for rows in range(1, df_rows):
# 					for columns in range(1, df_columns, 2):
# 						line = df.iloc[rows, columns]
# 						if line == "sg" or line == "pl":
# 							pass

# 						else:
# 							line = re.sub(r"(.+)", f"{stem}\\1", line)
# 							search_string = re.compile("\n", re.M)
# 							replace_string = " "
# 							matches = re.sub(search_string, replace_string, line)

# 							all_inflections += matches + " "
						
# 			except:
# 				with open("output/errors.txt", "a") as error_log:
# 					error_log.write(f"error on: {headword}\n")

dedupe and convert to df

In [ ]:
# all_inflections_list = all_inflections.split()
# all_inflections_list = list(dict.fromkeys(all_inflections_list))
# all_inflections_df = pd.DataFrame(all_inflections_list)
# all_inflections_df.to_csv("output/all_inflections_df.csv", index=False, header=False)

generate all inflection tables

In [ ]:
dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
dpd_df.fillna("", inplace=True) 
dpd_df_length = len(dpd_df)

row = 0

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	headword_clean = re.sub(" \d*$", "", headword)
	stem = dpd_df.loc[row, "Stem"]
	if stem == "*":
		stem = ""
	pattern = dpd_df.loc[row, "Pattern"]
	pos = dpd_df.loc[row, "POS"]
	metadata = dpd_df.loc[row, "Metadata"]
	meaning = dpd_df.loc[row, "Meaning IN CONTEXT"]

	

	if row % 2000 == 0:
		print(f"{row} {headword}")
	
	with open(f"output/html/{headword}.html", "w") as html_table:

		if stem == "-":
			html_table.write(f"<p><b>{headword_clean}</b> is indeclinable")

		elif stem == "!":
			html_table.write(f"<p>click on <b>{pattern}</b> for inflection table")

		else:
			df = pd.read_csv(f"output/patterns/{pattern}.csv", sep="\t", header=None)
			df.fillna("", inplace=True)
			df_rows = df.shape[0]
			df_columns = df.shape[1]

			for rows in range(1, df_rows):
				for columns in range(1, df_columns, 2):
					cell = df.iloc[rows, columns]

					if cell == "sg" or cell == "pl":
						pass

					else:
						cell = re.sub(r"(.+)", f"<b>\\1</b>", cell) # add bold
						cell = re.sub(r"(.+)", f"{stem}\\1", cell) # add stem
						cell = re.sub(r"\n", "<br>", cell) # add line breaks
						df.iloc[rows, columns] = cell
			
			for columns in range(2, df_columns, 2):	# drop extra rows
				df.drop([columns], axis=1, inplace=True) 
			
			df_rows = df.shape[0]
			df_columns = df.shape[1]

			html_file = open (f"output/html/{headword}.html", "w")
			for rows in range(df_rows):
				for columns in range(df_columns):
					cell = df.iloc[rows, columns]
					if rows == 0 and columns == 0:
						cell = """<table class = "table2"><tr><th></th>"""
						html_file.write(cell)
					if rows == 0 and columns > 0:
						cell = "<th>" + cell + "</th>"
						html_file.write(cell)
					if rows > 0 and columns == 0:
						cell = "<tr><th>" + cell + "</th>"
						html_file.write(cell)
					if rows > 0 and columns > 0:
						cell = "<td>" + cell + "</td>"
						html_file.write(cell)
					if rows == df_rows and columns == df_columns:
						cell = "<td>" + cell + "</td></table>"
						html_file.write(cell)
			html_file.close()


			# for columns in range(df_columns): # add table html
			# 	try:
			# 		if columns == 0:
			# 			df.iloc[0, columns] = """<table class = "table2"><tr><th>""" + df.iloc[0, columns] + "</th>"
			# 			df.iloc[1:, columns] =  "<tr><th>" + df.loc[1:, columns] + "</th>"
			# 		if columns > 0:
			# 			df.iloc[0, columns] = "<th>" + df.iloc[0, columns] + "</th>"
			# 			df.iloc[1:, columns] =  "<td>" + df.iloc[1:, columns] + "</td>"
			# 		if columns == df_columns:
			# 			df.iloc[df_rows, df_columns] =  df.iloc[df_rows, df_columns] + "</table>"

			# 	except:
			# 		with open("output/table errors.txt", "a") as table_error_log:
			# 			table_error_log.write(f"")
			# 			table_error_log.write(f"{headword=}\n{columns=}\n\n")
	
			# df.to_csv(f"output/html/{headword}.html",  sep=" ", index=None, header=None)



generate all synonyms

In [ ]:
dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
dpd_df.fillna("", inplace=True)
dpd_df_length = len(dpd_df)

all_synonyms = open ("output/all synonyms.csv", "w")

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	headword_clean = re.sub(" \d*$", "", headword)
	stem = dpd_df.loc[row, "Stem"]
	if stem == "*":
		stem = ""
	pattern = dpd_df.loc[row, "Pattern"]
	pos = dpd_df.loc[row, "POS"]
	metadata = dpd_df.loc[row, "Metadata"]
	meaning = dpd_df.loc[row, "Meaning IN CONTEXT"]

	synonyms = ""

	if row % 1000 == 0:
		print(f"{row} {headword}")

	if stem == "-":
		synonyms += headword_clean + " "

	elif stem == "!":
		synonyms += headword_clean + " "

	else:
		try:
			df = pd.read_csv(f"output/patterns/{pattern}.csv", sep="\t", header=None)
			df.fillna("", inplace=True)
			df_rows = df.shape[0]
			df_columns = df.shape[1]

			for rows in range(1, df_rows):
				for columns in range(1, df_columns, 2):
					line = df.iloc[rows, columns]
					if line == "sg" or line == "pl":
						pass

					else:
						line = re.sub(r"(.+)", f"{stem}\\1", line)
						search_string = re.compile("\n", re.M)
						replace_string = " "
						matches = re.sub(search_string, replace_string, line)

						synonyms += matches + " "
					
		except:
			with open("output/errors.txt", "a") as error_log:
				error_log.write(f"error on: {headword}\n")
	
	all_synonyms.write(f"{headword}	{synonyms}\n")

all_synonyms.close()


transcribe synonyms

In [2]:
all_synonyms = open("output/all synonyms.csv", "r")
all_synonyms_read = all_synonyms.read()
all_synonyms.close()

all_synonyms_translit = open("output/all synonyms translt.csv", "w")

sinhala = transliterate.process("IAST","Sinhala", all_synonyms_read, post_options =['SinhalaPali'])
devanagari = transliterate.process("IAST","Devanagari",all_synonyms_read)

roman = all_synonyms_read.split("\n")[:-1]
sinhala = sinhala.split("\n")
devanagari = devanagari.split("\n")

for i in zip(roman, sinhala, devanagari):	
  all_synonyms_translit.write(i[0]+i[1].split("\t")[1]+i[2].split("\t")[1]+"\n")

all_synonyms_translit.close()

export to pickle

In [3]:
all_synonyms = pd.read_csv("output/all synonyms translt.csv", header=None, sep="\t")

length = len(all_synonyms)

for row in range(length):
	headword = all_synonyms.iloc[row, 0]
	synonyms = str(all_synonyms.iloc[row, 1])

	synonyms_list = synonyms.split()
	synonyms_list = list(dict.fromkeys(synonyms_list))

	with open(f"output/synonyms/{headword}", "wb") as text_file:
		pickle.dump(synonyms_list, text_file)
